# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [93]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'WeatherPy/output_data/cities.csv'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat",
geo=True,
tiles='OSM',
size='Humidity',
color='City',
hover_cols=['City', 'Humidity'])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [54]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] < 300) & 
    (city_data_df['Max Temp'] > 250) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] ==0)
].dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,anadyr,64.7500,177.4833,277.43,87,0,2.00,RU,1688911862
48,48,finley,46.1540,-119.0339,298.30,48,0,3.13,US,1688911863
124,124,bryne,58.7354,5.6477,296.96,60,0,3.09,NO,1688911894
127,127,tiksi,71.6872,128.8694,276.12,67,0,1.99,RU,1688911894
132,132,puerto natales,-51.7236,-72.4875,266.40,93,0,3.60,CL,1688911895
145,145,newman,37.3138,-121.0208,287.34,76,0,1.89,US,1688911906
181,181,alesund,62.4723,6.1549,295.88,77,0,4.12,NO,1688911910
186,186,badger,64.8000,-147.5333,285.77,83,0,2.06,US,1688911910
204,204,port macquarie,-31.4333,152.9167,279.76,70,0,2.32,AU,1688911925
209,209,karratha,-20.7377,116.8463,293.25,29,0,3.33,AU,1688911925


### Step 3: Create a new DataFrame called `hotel_df`.

In [57]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
40,anadyr,RU,64.7500,177.4833,87,
48,finley,US,46.1540,-119.0339,48,
124,bryne,NO,58.7354,5.6477,60,
127,tiksi,RU,71.6872,128.8694,67,
132,puerto natales,CL,-51.7236,-72.4875,93,
145,newman,US,37.3138,-121.0208,76,
181,alesund,NO,62.4723,6.1549,77,
186,badger,US,64.8000,-147.5333,83,
204,port macquarie,AU,-31.4333,152.9167,70,
209,karratha,AU,-20.7377,116.8463,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [70]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lon=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
finley - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
xai-xai - nearest hotel: No hotel found
quilmana - nearest hotel: No hotel found
longonjo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
40,anadyr,RU,64.7500,177.4833,87,Гостевой дом
48,finley,US,46.1540,-119.0339,48,No hotel found
124,bryne,NO,58.7354,5.6477,60,Jæren hotell
127,tiksi,RU,71.6872,128.8694,67,Арктика
132,puerto natales,CL,-51.7236,-72.4875,93,Dorotea Patagonia Hostel
145,newman,US,37.3138,-121.0208,76,No hotel found
181,alesund,NO,62.4723,6.1549,77,Brosundet
186,badger,US,64.8000,-147.5333,83,Hotel North Pole
204,port macquarie,AU,-31.4333,152.9167,70,The Observatory
209,karratha,AU,-20.7377,116.8463,29,Karratha International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [80]:
%%capture --no-display
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", "Lat",
geo=True,
tiles='OSM',
size='Humidity',
color='City',
hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)